In [4]:
import SeasonData as sdata
import pandas as pd
import ipywidgets

In [ ]:
# Generating play-by-play data for year 2022
rs_data = sdata.NHLData()
rs_data.fetch_regular_season(year = 2022)
pbp_data = [pd.DataFrame(game['plays']) for game in rs_data.regular_season['2022']]

In [4]:
pbp_data[0].head()

,eventId,periodDescriptor,timeInPeriod,timeRemaining,situationCode,homeTeamDefendingSide,typeCode,typeDescKey,sortOrder,details
0,8,"{'number': 1, 'periodType': 'REG', 'maxRegulat...",00:00,20:00,1551,right,520,period-start,8,NaN
1,9,"{'number': 1, 'periodType': 'REG', 'maxRegulat...",00:00,20:00,1551,right,502,faceoff,9,"{'eventOwnerTeamId': 18, 'losingPlayerId': 847..."
2,51,"{'number': 1, 'periodType': 'REG', 'maxRegulat...",00:11,19:49,1551,right,503,hit,10,"{'xCoord': -31, 'yCoord': -36, 'zoneCode': 'D'..."
3,52,"{'number': 1, 'periodType': 'REG', 'maxRegulat...",00:17,19:43,1551,right,503,hit,11,"{'xCoord': -80, 'yCoord': 37, 'zoneCode': 'D',..."
4,10,"{'number': 1, 'periodType': 'REG', 'maxRegulat...",00:23,19:37,1551,right,506,shot-on-goal,12,"{'xCoord': 44, 'yCoord': 8, 'zoneCode': 'O', '..."


In [7]:
rs_data.regular_season['2022'][0]

{'id': 2022020001,
 'season': 20222023,
 'gameType': 2,
 'limitedScoring': False,
 'gameDate': '2022-10-07',
 'venue': {'default': 'O2 Czech Republic'},
 'venueLocation': {'default': 'Prague',
  'cs': 'Praha',
  'de': 'Prag',
  'fi': 'Praha',
  'sk': 'Praha',
  'sv': 'Prag'},
 'startTimeUTC': '2022-10-07T18:00:00Z',
 'easternUTCOffset': '-04:00',
 'venueUTCOffset': '+02:00',
 'tvBroadcasts': [{'id': 324,
   'market': 'N',
   'countryCode': 'US',
   'network': 'NHLN',
   'sequenceNumber': 23},
  {'id': 314,
   'market': 'A',
   'countryCode': 'US',
   'network': 'NBCSCA',
   'sequenceNumber': 63},
  {'id': 375,
   'market': 'H',
   'countryCode': 'US',
   'network': 'BSSO',
   'sequenceNumber': 70}],
 'gameState': 'OFF',
 'gameScheduleState': 'OK',
 'periodDescriptor': {'number': 3,
  'periodType': 'REG',
  'maxRegulationPeriods': 3},
 'awayTeam': {'id': 28,
  'name': {'default': 'Sharks'},
  'abbrev': 'SJS',
  'score': 1,
  'sog': 31,
  'logo': 'https://assets.nhle.com/logos/nhl/svg/SJ

In [8]:
pbp_data[0]

,eventId,periodDescriptor,timeInPeriod,timeRemaining,situationCode,homeTeamDefendingSide,typeCode,typeDescKey,sortOrder,details
0,8,"{'number': 1, 'periodType': 'REG', 'maxRegulat...",00:00,20:00,1551,right,520,period-start,8,NaN
1,9,"{'number': 1, 'periodType': 'REG', 'maxRegulat...",00:00,20:00,1551,right,502,faceoff,9,"{'eventOwnerTeamId': 18, 'losingPlayerId': 847..."
2,51,"{'number': 1, 'periodType': 'REG', 'maxRegulat...",00:11,19:49,1551,right,503,hit,10,"{'xCoord': -31, 'yCoord': -36, 'zoneCode': 'D'..."
3,52,"{'number': 1, 'periodType': 'REG', 'maxRegulat...",00:17,19:43,1551,right,503,hit,11,"{'xCoord': -80, 'yCoord': 37, 'zoneCode': 'D',..."
4,10,"{'number': 1, 'periodType': 'REG', 'maxRegulat...",00:23,19:37,1551,right,506,shot-on-goal,12,"{'xCoord': 44, 'yCoord': 8, 'zoneCode': 'O', '..."
...,...,...,...,...,...,...,...,...,...,...
318,699,"{'number': 3, 'periodType': 'REG', 'maxRegulat...",19:43,00:17,1551,right,516,stoppage,656,{'reason': 'puck-in-benches'}
319,1007,"{'number': 3, 'periodType': 'REG', 'maxRegulat...",19:43,00:17,1551,right,502,faceoff,657,"{'eventOwnerTeamId': 18, 'losingPlayerId': 847..."
320,700,"{'number': 3, 'periodType': 'REG', 'maxRegulat...",19:58,00:02,1551,right,503,hit,658,"{'xCoord': -2, 'yCoord': -36, 'zoneCode': 'N',..."
321,1008,"{'number': 3, 'periodType': 'REG', 'maxRegulat...",20:00,00:00,1551,right,521,period-end,659,NaN


In [9]:
rs_data.regular_season['2022'][0]

{'id': 2022020001,
 'season': 20222023,
 'gameType': 2,
 'limitedScoring': False,
 'gameDate': '2022-10-07',
 'venue': {'default': 'O2 Czech Republic'},
 'venueLocation': {'default': 'Prague',
  'cs': 'Praha',
  'de': 'Prag',
  'fi': 'Praha',
  'sk': 'Praha',
  'sv': 'Prag'},
 'startTimeUTC': '2022-10-07T18:00:00Z',
 'easternUTCOffset': '-04:00',
 'venueUTCOffset': '+02:00',
 'tvBroadcasts': [{'id': 324,
   'market': 'N',
   'countryCode': 'US',
   'network': 'NHLN',
   'sequenceNumber': 23},
  {'id': 314,
   'market': 'A',
   'countryCode': 'US',
   'network': 'NBCSCA',
   'sequenceNumber': 63},
  {'id': 375,
   'market': 'H',
   'countryCode': 'US',
   'network': 'BSSO',
   'sequenceNumber': 70}],
 'gameState': 'OFF',
 'gameScheduleState': 'OK',
 'periodDescriptor': {'number': 3,
  'periodType': 'REG',
  'maxRegulationPeriods': 3},
 'awayTeam': {'id': 28,
  'name': {'default': 'Sharks'},
  'abbrev': 'SJS',
  'score': 1,
  'sog': 31,
  'logo': 'https://assets.nhle.com/logos/nhl/svg/SJ

In [37]:
# Fetching teams abbreviation
print('game id')
print(rs_data.regular_season['2022'][0]['id'])
print('away team abbrev')
print(rs_data.regular_season['2022'][0]['awayTeam']['abbrev'])
print('home team abbrev')
print(rs_data.regular_season['2022'][0]['homeTeam']['abbrev'])
print('away team score')
print(rs_data.regular_season['2022'][0]['awayTeam']['score'])
print('home team score')
print(rs_data.regular_season['2022'][0]['homeTeam']['score'])
print('away team shoot on goal')
print(rs_data.regular_season['2022'][0]['awayTeam']['sog'])
print('home team shoot on goal')
print(rs_data.regular_season['2022'][0]['homeTeam']['sog'])


game id
2022020001
away team abbrev
SJS
home team abbrev
NSH
away team score
1
home team score
4
away team shoot on goal
31
home team shoot on goal
32


In [11]:
# we assume it is a regular season

season = '2022'
rs_data = sdata.NHLData()
rs_data.fetch_regular_season(year = season)

def plot_game(game_number):    
    game_id = int(f"{season}{sdata.GameType.REGULAR_SEASON.value}{game_number:04d}")

    # to improve
    game_data = next((item for item in rs_data.regular_season[season] if item["id"] == game_id), None)
    
    print(game_data['startTimeUTC'])
    print(f"Game ID: {game_number}; {game_data['homeTeam']['abbrev']} (home) vs {game_data['awayTeam']['abbrev']} (away)")
    
    col1 = ['', 'Teams', 'Goals', 'SoG']
    col2 = ["Home",f"{game_data['homeTeam']['abbrev']}", f"{game_data['homeTeam']['score']}", f"{game_data['homeTeam']['sog']}"]
    col3 = ["Away",f"{game_data['awayTeam']['abbrev']}", f"{game_data['awayTeam']['score']}", f"{game_data['awayTeam']['sog']}"]
    print('')
    for c1, c2, c3 in zip(col1, col2, col3):
        print(f'{c1:<18} {c2:<18} {c3:<18}')

        
ipywidgets.interact(plot_game, game_number=(1,len(rs_data.regular_season[season]),1))

Data imported: 0


interactive(children=(IntSlider(value=656, description='game_number', max=1312, min=1), Output()), _dom_classe…

<function __main__.plot_game(game_number)>